# About

This is my attempt for <b>early data cleaning</b> process on the `rawdata.csv` without doing further analysis. 

Let me start by import pandas library to do the data cleaning process.

In [1]:
import pandas as pd

Process further by loading the `rawdata.csv` and check whether the loading is fined.

In [2]:
data = pd.read_csv("../dataset/rawdata.csv")
data.head()

,Title,Time_Created,Category,Teaser
0,Q and A with senior minister Ismail Sabri,"May 7, 2020 @ 9:00am",EXCLUSIVE,KUALA LUMPUR: Datuk Seri Ismail Sabri Yaakob h...
1,MCO-linked domestic violence rises,"Apr 4, 2020 @ 12:01pm",EXCLUSIVE,KUALA LUMPUR: AS the number of people being co...
2,Victims living a nightmare with no escape,"Apr 4, 2020 @ 12:00pm",EXCLUSIVE,KUALA LUMPUR: The Covid-19 pandemic and ensuin...
3,NST Exclusive: Keeping the memory of MH17 vict...,"Mar 8, 2020 @ 11:25am",EXCLUSIVE,AMSTERDAM: On a vast expanse of green not far ...
4,KL's iconic buildings to get facelift,"Feb 3, 2020 @ 12:13pm",EXCLUSIVE,KUALA LUMPUR: The decaying century-old cluster...


I start investigating the data type of the feature.

In [3]:
data.dtypes

Title           object
Time_Created    object
Category        object
Teaser          object
dtype: object

I found out that the feature `Time_Created` is in the form of the <b>string</b>. Therefore, it is neccessary to convert
that feature to <b>datetime</b> data type to make sure that I can perform time series analysis in future. 

## Part I: Checking and Dropping duplicate data

Let me have a quick check whether there is any duplicate data.

In [4]:
sum(data.duplicated())

118

It seem like there are 118 duplicated rows.

Check what is duplicated index in the raw dataset.

In [5]:
list(data[data.duplicated()].index)

[20,
 40,
 60,
 80,
 100,
 120,
 140,
 160,
 180,
 200,
 220,
 240,
 260,
 280,
 300,
 320,
 340,
 360,
 380,
 400,
 420,
 440,
 460,
 480,
 520,
 540,
 560,
 580,
 600,
 620,
 640,
 660,
 680,
 700,
 720,
 740,
 760,
 780,
 800,
 820,
 840,
 860,
 880,
 900,
 920,
 940,
 960,
 980,
 1020,
 1040,
 1060,
 1080,
 1100,
 1120,
 1140,
 1160,
 1180,
 1200,
 1220,
 1240,
 1260,
 1280,
 1300,
 1320,
 1340,
 1360,
 1380,
 1400,
 1420,
 1440,
 1460,
 1480,
 1520,
 1540,
 1560,
 1580,
 1600,
 1620,
 1640,
 1660,
 1680,
 1700,
 1720,
 1740,
 1760,
 1780,
 1800,
 1820,
 1840,
 1860,
 1880,
 1900,
 1920,
 1940,
 1960,
 1980,
 2020,
 2040,
 2060,
 2080,
 2100,
 2120,
 2140,
 2160,
 2180,
 2200,
 2220,
 2240,
 2260,
 2300,
 2320,
 2340,
 2380,
 2400,
 2420,
 2440,
 2460,
 2480]

When I go back to the <a href="https://www.nst.com.my/">NST website</a>,
I found out that the <b>last element</b> in this <a href="https://www.nst.com.my/news/exclusive?page=0"> page</a> is same as
the <b>first element</b> of this <a href="https://www.nst.com.my/news/exclusive?page=1"> page</a>. The web scrapping that I had
created do not escape this duplication.

Keep the first element of every duplicated row, drop the others.

In [6]:
print("Before: ", len(data))
data = data.drop_duplicates(keep="first")
print("After: ", len(data))

Before:  2500
After:  2382


<b>Part I</b> completed.

## Part II: Front Part Teaser Cleaning

In [7]:
data.head()

,Title,Time_Created,Category,Teaser
0,Q and A with senior minister Ismail Sabri,"May 7, 2020 @ 9:00am",EXCLUSIVE,KUALA LUMPUR: Datuk Seri Ismail Sabri Yaakob h...
1,MCO-linked domestic violence rises,"Apr 4, 2020 @ 12:01pm",EXCLUSIVE,KUALA LUMPUR: AS the number of people being co...
2,Victims living a nightmare with no escape,"Apr 4, 2020 @ 12:00pm",EXCLUSIVE,KUALA LUMPUR: The Covid-19 pandemic and ensuin...
3,NST Exclusive: Keeping the memory of MH17 vict...,"Mar 8, 2020 @ 11:25am",EXCLUSIVE,AMSTERDAM: On a vast expanse of green not far ...
4,KL's iconic buildings to get facelift,"Feb 3, 2020 @ 12:13pm",EXCLUSIVE,KUALA LUMPUR: The decaying century-old cluster...


The front part for the most of the teaser starts with the location. Since my project's objective is doing news classification,
it is reasonable to clean up the teaser that starts with location. 
<b>Location should not have any relations to the classification.</b>

Initialise a mask for data that contains semicolon `:`.

In [8]:
contain_semi_colon_mask = data.Teaser.str.contains(":") == True

Initialise a mask for first element of the data after split with semicolon that contains all upper character 
(Indicating the location).

In [9]:
contain_first_place_mask = data.Teaser.apply(lambda x: str(x).split(":")[0]).str.isupper() == True

Interception between the two masks clean up the front part of the teaser.

In [10]:
data.loc[contain_semi_colon_mask & contain_first_place_mask, "Teaser"] = data.Teaser.apply(lambda x: " ".join(str(x).split(":")[1 :]))

In [11]:
data.head()

,Title,Time_Created,Category,Teaser
0,Q and A with senior minister Ismail Sabri,"May 7, 2020 @ 9:00am",EXCLUSIVE,Datuk Seri Ismail Sabri Yaakob has been in th...
1,MCO-linked domestic violence rises,"Apr 4, 2020 @ 12:01pm",EXCLUSIVE,AS the number of people being confined to the...
2,Victims living a nightmare with no escape,"Apr 4, 2020 @ 12:00pm",EXCLUSIVE,The Covid-19 pandemic and ensuing Movement Co...
3,NST Exclusive: Keeping the memory of MH17 vict...,"Mar 8, 2020 @ 11:25am",EXCLUSIVE,On a vast expanse of green not far from the A...
4,KL's iconic buildings to get facelift,"Feb 3, 2020 @ 12:13pm",EXCLUSIVE,The decaying century-old cluster of colonial ...


<b>Part II</b> completed.

## Part III: Dropping the Rows that do not Contain Teaser

Find out the total number of rows that do not contain teaser `NA`.

In [12]:
sum(data.Teaser.isna())

30

There are some rows do not contain teaser.

In [13]:
print("Before dropping Na: ", len(data))
data = data.dropna(subset=["Teaser"])
print("After dropping Na: ", len(data))

Before dropping Na:  2382
After dropping Na:  2352


<b>Part III</b> completed.

## Part IV: Transform Contractions to Full English Text

Initialise the contrations dictionary. I found out 
this <a href="https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python"> page</a> 
has most of the contractions that I need to clean up the raw data. 

In [14]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't've": "cannot have",
"can't": "cannot",
"could've": "could have",
"couldn't've": "could not have",
"couldn't": "could not",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't've": "had not have",
"hadn't": "had not",
"hasn't": "has not",
"haven't": "have not",
"he'd've": "he would have",
"he'd": "he had",
"he'll've": "he will have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd've": "I would have",
"I'd": "I would",
"I'll've": "I will have",
"I'll": "I will",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd've": "it would have",
"it'd": "it would",
"it'll've": "it will have",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't've": "might not have",
"mightn't": "might not",
"must've": "must have",
"mustn't've": "must not have",
"mustn't": "must not",
"needn't've": "need not have",
"needn't": "need not",
"o'clock": "of the clock",
"oughtn't've": "ought not have",
"oughtn't": "ought not",
"shan't've": "shall not have",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd've": "she would have",
"she'd": "she had",
"she'll've": "she will have",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't've": "should not have",
"shouldn't": "should not",
"so've": "so have",
"so's": "so is",
"that'd've": "that would have",
"that'd": "that had",
"that's": "that is",
"there'd've": "there would have",
"there'd": "there would",
"there's": "there is",
"they'd've": "they would have",
"they'd": "they would",
"they'll've": "they will have",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd've": "we would have",
"we'd": "we would",
"we'll've": "we will have",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll've": "what will have",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll've": "who will have",
"who'll": "who will",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't've": "will not have",
"won't": "will not",
"would've": "would have",
"wouldn't've": "would not have",
"wouldn't": "would not",
"y'all'd've": "you all would have",
"y'all'd": "you all would",
"y'all're": "you all are",
"y'all've": "you all have",
"y'all": "you all",
"you'd've": "you would have",
"you'd": "you would",
"you'll've": "you will have",
"you'll": "you will",
"you're": "you are",
"you've": "you have"
}

Replace the contractions with the full english text.

In [15]:
for element in contractions.keys():
    data.loc[:, "Title"] = data.Title.str.replace(element, contractions[element])
    data.loc[:, "Teaser"] = data.Teaser.str.replace(element, contractions[element])

<b>Part IV</b> completed.

## Part V: Format the category.

Find out what is the available news categories.

In [16]:
data["Category"].unique()

array(['EXCLUSIVE', 'CRIME & COURTS', 'NATION',
       'GOVERNMENT / PUBLIC POLICY', 'POLITICS'], dtype=object)

There are 5 categories that all are full capital letter. It's time to clean it up.

In [17]:
data.loc[data["Category"] == "EXCLUSIVE", "Category"] = "Exclusive"
data.loc[data["Category"] == "CRIME & COURTS", "Category"] = "Crime and Courts"
data.loc[data["Category"] == "NATION", "Category"] = "Nation"
data.loc[data["Category"] == "GOVERNMENT / PUBLIC POLICY", "Category"] = "Government or Public Policy"
data.loc[data["Category"] == "POLITICS", "Category"] = "Politics"

Check whether the cleaning is done correctly.

In [18]:
data["Category"].unique()

array(['Exclusive', 'Crime and Courts', 'Nation',
       'Government or Public Policy', 'Politics'], dtype=object)

<b>Part V</b> completed.

Exclude the the feature `time_created` and check the head of the dataset.

In [19]:
data_without_time = data[["Title", "Category", "Teaser"]]
data_without_time.head()

,Title,Category,Teaser
0,Q and A with senior minister Ismail Sabri,Exclusive,Datuk Seri Ismail Sabri Yaakob has been in th...
1,MCO-linked domestic violence rises,Exclusive,AS the number of people being confined to the...
2,Victims living a nightmare with no escape,Exclusive,The Covid-19 pandemic and ensuing Movement Co...
3,NST Exclusive: Keeping the memory of MH17 vict...,Exclusive,On a vast expanse of green not far from the A...
4,KL's iconic buildings to get facelift,Exclusive,The decaying century-old cluster of colonial ...


Save the dataset as `early_data_cleaning_without_time.csv` for the eda and further cleaning.

In [20]:
data_without_time.to_csv("../dataset/early_data_cleaning_without_time.csv", index=False)

## Extra:

The following part is my attempt to obtain the dataset with time_created feature. 

## Extra I: Drop the Rows that do not Contain Exact Time

Find out the number of rows that contains the word not related to exact time. <br/>
<b>E.g.: 5 hours ago</b>

In [21]:
sum(data["Time_Created"].apply(lambda x: "ago" in x))

53

Drop the rows and check the validity based on the number of rows remaining.

In [22]:
time_masker = data["Time_Created"].apply(lambda x: "ago" not in x)
print("Before dropping using time mask: ", len(data))
data_with_time = data[time_masker]
print("After dropping using time mask: ", len(data))

Before dropping using time mask:  2352
After dropping using time mask:  2352


<b>Extra I</b> completed.

## Extra II: Convert the Feature Time_Created from String to datetime

Create a brand new DataFrame and save the appropriate feature based on the splitting result from the feature `Time_Created`.

In [23]:
time = pd.DataFrame()
time["Time"] = data_with_time["Time_Created"].apply(lambda x: x.split(" ")[-1])
time['Day'] = data_with_time["Time_Created"].apply(lambda x: x.split(" ")[1].replace(",", ""))
time['Month'] = data_with_time["Time_Created"].apply(lambda x: x.split(" ")[0])
time["Year"] = data_with_time["Time_Created"].apply(lambda x: x.split(" ")[2])
time.head()

,Time,Day,Month,Year
0,9:00am,7,May,2020
1,12:01pm,4,Apr,2020
2,12:00pm,4,Apr,2020
3,11:25am,8,Mar,2020
4,12:13pm,3,Feb,2020


Format the `day` and check the results.

In [24]:
def day_formater(x):
    if len(x) == 1:
        return str(0) + x
    else:
        return x

time["Day"] = time["Day"].apply(day_formater)
time.tail()

,Time,Day,Month,Year
2495,2:59pm,26,Oct,2020
2496,1:29pm,26,Oct,2020
2497,1:03pm,26,Oct,2020
2498,12:28pm,26,Oct,2020
2499,10:11am,26,Oct,2020


Format the `month` and check the results.

In [25]:
month_dict = {"Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06", "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"}

for month in month_dict.keys():
    time.loc[time["Month"] == month, "Month"] = month_dict[month]

time.head()

,Time,Day,Month,Year
0,9:00am,07,05,2020
1,12:01pm,04,04,2020
2,12:00pm,04,04,2020
3,11:25am,08,03,2020
4,12:13pm,03,02,2020


Format the `hours/minutes` and check the results.

In [26]:
def hour_formater(x):
    dirty_time_list = x.split(":")
    hour = dirty_time_list[0]
    if "am" in x:
        if int(hour) < 10:
            return str(0) + hour
        else:
            return hour
    else:
        if int(hour) <= 11:
            return str(int(hour) + 12)
        else:
            return hour
        
time["Hour"] = time["Time"].apply(hour_formater)
time["Minute"] = time["Time"].apply(lambda x: x.split(":")[1].replace("am", "").replace("pm", ""))
time.head()

,Time,Day,Month,Year,Hour,Minute
0,9:00am,07,05,2020,09,00
1,12:01pm,04,04,2020,12,01
2,12:00pm,04,04,2020,12,00
3,11:25am,08,03,2020,11,25
4,12:13pm,03,02,2020,12,13


Creating the final version of the time and check the final result.

In [27]:
time["Final"] = time["Year"] + "-" + time["Month"] + "-" + time["Day"] + " " + time["Hour"] + ":" + time["Minute"]
time.head()

,Time,Day,Month,Year,Hour,Minute,Final
0,9:00am,07,05,2020,09,00,2020-05-07 09:00
1,12:01pm,04,04,2020,12,01,2020-04-04 12:01
2,12:00pm,04,04,2020,12,00,2020-04-04 12:00
3,11:25am,08,03,2020,11,25,2020-03-08 11:25
4,12:13pm,03,02,2020,12,13,2020-02-03 12:13


Covert the `string` format to `datetime` format and check the type.

In [28]:
time["Final"] = pd.to_datetime(time["Final"])
time["Final"]

0      2020-05-07 09:00:00
1      2020-04-04 12:01:00
2      2020-04-04 12:00:00
3      2020-03-08 11:25:00
4      2020-02-03 12:13:00
               ...        
2495   2020-10-26 14:59:00
2496   2020-10-26 13:29:00
2497   2020-10-26 13:03:00
2498   2020-10-26 12:28:00
2499   2020-10-26 10:11:00
Name: Final, Length: 2299, dtype: datetime64[ns]

Assign the final time format to the original `Time_Created` feature.

In [29]:
pd.options.mode.chained_assignment = None
data_with_time.loc[:,"Time_Created"] = time["Final"]
data_with_time.head()

,Title,Time_Created,Category,Teaser
0,Q and A with senior minister Ismail Sabri,2020-05-07 09:00:00,Exclusive,Datuk Seri Ismail Sabri Yaakob has been in th...
1,MCO-linked domestic violence rises,2020-04-04 12:01:00,Exclusive,AS the number of people being confined to the...
2,Victims living a nightmare with no escape,2020-04-04 12:00:00,Exclusive,The Covid-19 pandemic and ensuing Movement Co...
3,NST Exclusive: Keeping the memory of MH17 vict...,2020-03-08 11:25:00,Exclusive,On a vast expanse of green not far from the A...
4,KL's iconic buildings to get facelift,2020-02-03 12:13:00,Exclusive,The decaying century-old cluster of colonial ...


<b>Extra II</b> completed.

Save the dataset as `early_data_cleaning_with_time.csv` for future use.

In [30]:
data_with_time.to_csv("../dataset/early_data_cleaning_with_time.csv", index=False)

<b>Early Data Cleaning</b> process completed